# Setup

### additional dependencies/configuration

In [ ]:
!pip install geopandas --user -q

### modules

In [1]:
from functools import partial
import pyproj
from pyspark import SparkContext
from geopyspark.geotrellis.color import ColorMap, get_colors_from_matplotlib
from geopyspark.geotrellis.tms import TMS
from geonotebook.wrappers import TMSRasterData
from shapely.geometry import MultiPolygon, shape
from shapely.ops import transform
import geopyspark as gps
import geopandas as gpd
import fiona

In [2]:
from overlay.data import data, mask
from overlay import analysis as an

### config spark

In [3]:
conf=gps.geopyspark_conf(appName="ClimateOverlay")
conf.set('spark.master.memory', '1200M')
sc = SparkContext(conf = conf)

In [4]:
hou = an.Overlay('houston')

In [5]:
hou.set_study_area('/home/hadoop/notebooks/data/read/boundaries/houston.shp')

# subsetting inundation raster

In [6]:
# polygon mask
# msas = gpd.read_file('/home/hadoop/notebooks/data/read/boundaries/tl_2015_us_cbsa.shp')
# nola = msas[msas['NAME'].str.contains('New Orleans', case = False)]

In [7]:
hou.study_area['bbox']

[-96.621979999999994, 28.764837, -94.353382999999994, 30.630284]

In [9]:
# clip raster
fulltiff = '/home/hadoop/notebooks/data/read/overlay-layers/flood/US_Category5_MOM_Inundation_HighTide.tif'
maskedtiff = '/home/hadoop/notebooks/data/write/overlay-layers/flood/houston_flood_category5.tif'
# mask.maskRasterFromBbox(fulltiff, hou.study_area['bbox'], maskedtiff)

# visualizing inundation raster

In [10]:
inund_pyramided = an.Overlay.get_pyramid(maskedtiff)

In [ ]:
# inund = gps.geotiff.get(
#     layer_type='spatial', 
#     uri=maskedtiff)

In [ ]:
# inund_tiled = inund.tile_to_layout(
#     layout=gps.GlobalLayout(), 
#     target_crs=3857)
# inund_pyramided = inund_tiled.pyramid().cache()

In [11]:
inund_histo         = inund_pyramided.get_histogram()
inund_colors        = get_colors_from_matplotlib('viridis', 100)
inund_color_map     = gps.ColorMap.from_histogram(inund_histo, inund_colors)

In [12]:
inund_tms = gps.TMS.build(inund_pyramided, inund_color_map)

In [17]:
M.set_center(hou.study_area['centroid']['x'], hou.study_area['centroid']['y'], 9)

In [18]:
M.add_layer(TMSRasterData(inund_tms))

### sea level rise

In [ ]:
# download and unzip sea level rise polygons
u = "https://coast.noaa.gov/htdata/Inundation/SLR/SLRdata/LA/LA_3_slr_data_dist.zip"
f = "/tmp/slr.zip"
data.sea_level_rise(u, f)

In [ ]:
help(fiona.open)

In [ ]:
# for i, name in enumerate (fiona.listlayers(sourceDatabaseFile)):
#     with fiona.open(sourceDatabaseFile, layer=1) as gdb:
#         print("Current layer's [", name, "] feature count is: ", len(gdb))

In [ ]:
t = fiona.open(sourceDatabaseFile, layer=1)

In [ ]:
gdb = '/tmp/slr/LA_3_slr_final_dist_polys.gdb'
with fiona.open(gdb, layer=1) as source:
    slr_crs = source.crs['init']
    slr = [MultiPolygon(shape(poly['geometry'])) for poly in source]

In [ ]:
def create_partial_reprojection_func(crs):
    return partial(pyproj.transform,
                   pyproj.Proj(init=crs),
                   pyproj.Proj(init='epsg:3857'))

reprojected_slr = [transform(create_partial_reprojection_func(slr_crs), s) for s in slr]

In [ ]:
# Rasterize slr polygons
rasterize_options = gps.RasterizerOptions(
    includePartial=True, sampleType='PixelIsArea')

slr_raster = gps.rasterize(
    geoms=reprojected_slr,
    crs="EPSG:3857",
    zoom=12,
    fill_value=1,
    cell_type=gps.CellType.FLOAT32,
    options=rasterize_options,
    num_partitions=50)

slr_pyramid = slr_raster.pyramid()

In [ ]:
slr_histo         = slr_pyramid.get_histogram()
slr_colors        = get_colors_from_matplotlib('viridis', 100)
slr_color_map     = gps.ColorMap.from_histogram(slr_histo, slr_colors)

In [ ]:
slr_tms = gps.TMS.build(slr_pyramid, slr_color_map)